In [1]:
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from arcgis.features import FeatureLayer

mhm_url = "https://services4.arcgis.com/feSRYIeirl5nUF92/arcgis/rest/services/MHM_2020/FeatureServer/0"

layer = FeatureLayer(mhm_url)

df = layer.query(where = "protocol = 'mosquito_habitat_mapper'").sdf
data = pd.DataFrame()

In [2]:
from arcgis.gis import GIS
gis = GIS(url="https://igestrategies.maps.arcgis.com", username="", password = "")

In [3]:
def process_data(threshold):
    global df, data
    suspect_df = df.groupby(by=['measuredDate','latitude','mosquitohabitatmapperWaterSou_2','siteName','longitude']).filter(lambda x: len(x) > threshold)

    # identify groups
    suspect_groups = suspect_df.groupby(by = ['measuredDate','latitude','mosquitohabitatmapperWaterSou_2','siteName','longitude'])

    # create a groups dataset
    groups_data = pd.DataFrame()
    group_num = 1 
    for group, temp in suspect_groups:
        new_df = temp.copy(True)
        new_df["Group Data"] = str(group)
        new_df["Group Name"] = str(group_num)
        
        group_num += 1
        groups_data = groups_data.append(new_df, ignore_index = True)
    data = groups_data

process_data(10)

In [4]:
from arcgis.features import FeatureLayerCollection
import geopandas as gpd
from arcgis.features import GeoAccessor
from IPython.display import display, clear_output
import ipywidgets as widgets

from arcgis.features import SpatialDataFrame
from arcgis.geoenrichment import enrich
from arcgis.raster import ImageryLayer
from arcgis.geometry import Geometry

map = gis.map('MHM Groups')
groups_features = pd.DataFrame()

def update_group_data():
    global data
    global groups_features
    geojson_data = data.drop(columns = ["SHAPE"])

    gdf = gpd.GeoDataFrame(
        geojson_data, geometry=gpd.points_from_xy(geojson_data.longitude, geojson_data.latitude))
    gdf.mosquitohabitatmapperComments.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
    groups_features = GeoAccessor.from_geodataframe(gdf)
    
def update_map():
    map.remove_layers()
    try:
        update_group_data()
        groups_features.spatial.plot(map_widget = map)
        
    except KeyError:
        print("no such group exists")
    map.basemap = 'dark-gray-vector'
    
update_map()

In [5]:
group_name = ""
def move_to_group(group):
    group_data = groups_features[groups_features["Group Name"] == group]
    latitude = group_data["latitude"].iloc[0]
    longitude = group_data["longitude"].iloc[0]
    map.center = [latitude, longitude]
    return group_data
    
def select_group(group):
    global group_name 
    group_name = group
    group_data = move_to_group(group)
    display(group_data)


In [6]:
def get_groups(threshold = 10):
    process_data(threshold)
    update_map()
    display(interactive(select_group, group = groups_features["Group Name"].unique()))
    


threshold_modifier = interactive(get_groups, threshold = (0, 200))

In [7]:
import os
import glob
import re
import requests
import sys
from IPython.display import Image


def show_images(file):
    try:
        display(Image("Images/" + file))
    except TypeError:
        print("No Pictures to download")

def download_picture(url, directory):
    def get_picture(file_name):
        downloaded_obj = requests.get(url, allow_redirects=True)
        parent_dir = os.path.join(directory, file_name)
        with open(parent_dir, "wb") as file:
            file.write(downloaded_obj.content)

    if "https://" in url:
        photo_id = re.search(r'(?<=\d\d\d\d\/\d\d\/\d\d\/).*(?=\/)', url).group(0)
        file_name = f"{photo_id}.jpg".replace(":","-")
        get_picture(file_name)

        
def get_pictures(group):
    files = glob.glob('Images/*')
    for f in files:
        os.remove(f)

    group_water_source = group["mosquitohabitatmapperWaterSou_1"].dropna().tolist()
    group_abdomen = group["mosquitohabitatmapperAbdomenClo"].dropna().tolist()
    group_larvae = group["mosquitohabitatmapperLarvaFullB"].dropna().tolist()
    group_urls = group_water_source + group_abdomen + group_larvae
    dl = 0
    total_length = len(group_urls)
    for urls in group_urls:
        if urls:
            dl += 1
            urls = urls.split(";")
            for url in urls:
                download_picture(url, "Images")
        
        done = int(50 * dl / total_length)
        sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (50-done)) )    
        sys.stdout.flush()
        



def download_photos(_):
    # "linking function with output"
    with out:
        # what happens when we press the button
        clear_output()
        print("Downloading Photos:")
        get_pictures(groups_features[groups_features["Group Name"] == group_name])
        print(' Finished Downloading!')
        
        photo_displayer = interactive(show_images, file=os.listdir('Images/')) 
        display(photo_displayer)

In [8]:
button = widgets.Button(description='Get Photos')
out = widgets.Output()

# linking button and function together using a button's method
button.on_click(download_photos)
# displaying button and its output together
button = widgets.VBox([button,out])

In [9]:
tabs = widgets.Tab(children = [threshold_modifier, button])

tabs.set_title(0, "Threshold Selection")
tabs.set_title(1, "Photo Downloader")


update_map()


display(map)
display(tabs)

MapView(layout=Layout(height='400px', width='100%'))